In [1]:
import tensorflow as tf
import os
from tqdm import tqdm
import urllib.request as req

In [2]:
# ! pip install tqdm

In [3]:
SOURCE_URL = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip"

In [4]:
DESTINATION = "data"
os.makedirs(DESTINATION, exist_ok=True)
data_file = "data.zip"

DESTINATION_ZIP_PATH = os.path.join(DESTINATION, data_file)

In [ ]:
filename, headers = req.urlretrieve(SOURCE_URL, DESTINATION_ZIP_PATH)

In [ ]:
filename

In [ ]:
print(headers)

In [ ]:
from zipfile import ZipFile

with ZipFile(filename, "r") as zip_f:
    zip_f.extractall("data/")

In [ ]:
TARGET_DATA_DIRS = ["Cat", "Dog"]
PARENT_DIR = os.path.join("data","PetImages")

In [ ]:
# !pip install Pillow

In [ ]:
from PIL import Image

In [ ]:
BAD_DATA_DIR = "bad_data"
os.makedirs(BAD_DATA_DIR, exist_ok=True)

In [ ]:
img = Image.open(r"C:\jay\Deep Learning classes\CNN_Classifier\notebook\data\PetImages\Dog\4.jpg")
img.format

In [ ]:
if img.format in ['JPEG', 'PNG', 'GIF', 'BMP']:
    print('hello')

In [ ]:
import shutil
import imghdr

In [ ]:
for dirs in os.listdir(PARENT_DIR):
    full_path_data_dir = os.path.join(PARENT_DIR, dirs)
    for imgs in os.listdir(full_path_data_dir):
        path_to_img = os.path.join(full_path_data_dir, imgs)
        try:
            img = Image.open(path_to_img)
            img.verify()
                
#             if len(img.getbands()) !=3 or imghdr.what(path_to_img) not in ['jpeg','png']:
#                 bad_data_path = os.path.join(BAD_DATA_DIR, imgs)
#                 shutil.move(path_to_img, bad_data_path)
#                 continue
                
#             print(f"{path_to_img} is verified with format {imghdr.what(path_to_img)}")
        except Exception as e:
            print(f"{path_to_img} is bad !")
            bad_data_path = os.path.join(BAD_DATA_DIR, imgs)
            shutil.move(path_to_img, bad_data_path)

In [ ]:
IMG_SIZE = (180, 180)
BATCH_SIZE = 32

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    PARENT_DIR,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    PARENT_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

In [ ]:
### Create a log dir for tensorboard logs

import time

def get_log_path(base_log_dir=os.path.join("logs","fit")):
    uniqueName = time.asctime().replace(" ", "_").replace(":", "")
    log_path = os.path.join(base_log_dir, uniqueName)
    print(f"saving logs at: {log_path}")
    return log_path

log_dir = get_log_path()

In [ ]:
file_writer = tf.summary.create_file_writer(logdir=log_dir)

In [ ]:
for images, labels in train_ds.take(1):
    print(images.shape, labels)

In [ ]:
import numpy as np

In [ ]:
with file_writer.as_default():
    images = np.array(images) ### (20, 28, 28, 1)

    tf.summary.image("samples", images.astype("uint8"), max_outputs=25, step=0)